In [5]:
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

data_dir = 'data' #data from polybox (ASV, taxonomy analysis)
or_dir = '../data'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

In [2]:
! wget -nv -O $data_dir/taxonomy_classification.qzv 'https://polybox.ethz.ch/index.php/s/c3rjz7z9GBUXvE8/download'
! wget -nv -O $data_dir/taxonomy_classification.qza 'https://polybox.ethz.ch/index.php/s/X8AoIryeOnLaCn5/download'
! wget -nv -O $data_dir/PJNB_dada2_table-filtered.qza 'https://polybox.ethz.ch/index.php/s/q2dsg2JxeJz3hko/download'
! wget -nv -O $data_dir/PJNB_dada2_rep_set-filtered.qza 'https://polybox.ethz.ch/index.php/s/vtDJzaCrmebL2u7/download'


2022-11-15 17:48:55 URL:https://polybox.ethz.ch/index.php/s/c3rjz7z9GBUXvE8/download [1967910/1967910] -> "data/taxonomy_classification.qzv" [1]
2022-11-15 17:48:55 URL:https://polybox.ethz.ch/index.php/s/X8AoIryeOnLaCn5/download [397957/397957] -> "data/taxonomy_classification.qza" [1]
2022-11-15 17:48:56 URL:https://polybox.ethz.ch/index.php/s/q2dsg2JxeJz3hko/download [1818719/1818719] -> "data/PJNB_dada2_table-filtered.qza" [1]
2022-11-15 17:48:56 URL:https://polybox.ethz.ch/index.php/s/vtDJzaCrmebL2u7/download [552613/552613] -> "data/PJNB_dada2_rep_set-filtered.qza" [1]


In [6]:
metadata = pd.read_csv(or_dir + '/metadata.tsv', sep = '\t')

In [10]:
print(metadata.shape)
print(metadata['host_id'].nunique())

(1669, 26)
80


In [33]:
taxonomic_assignments = pd.read_csv(data_dir+'/taxonomic_assignments.csv', index_col = 0)
#print(taxonomic_assignments.head())
print(taxonomic_assignments['diet_weaning'].unique())
print(taxonomic_assignments['diet_milk'].unique())

[nan True False]
[nan 'bd' 'fd' 'no milk']


In [32]:
col_names = taxonomic_assignments.columns
print(col_names[-25:])
col_names_species = col_names[0:-25] #remove metadata
print(len(col_names_species))
col_names_species = [i for i in col_names_species if not i.endswith('__')] #remove species unclassfied until species
print(len(col_names_species))

Index(['Library Layout', 'Instrument', 'collection_date', 'geo_location_name',
       'geo_latitude', 'geo_longitude', 'host_id', 'age_days', 'weight_kg',
       'length_cm', 'WAZ', 'HAZ', 'WHZ', 'BMI', 'birth_weight_g',
       'birth_length_cm', 'sex', 'delivery_mode', 'zygosity', 'race',
       'ethnicity', 'delivery_preterm', 'diet_milk', 'diet_weaning',
       'age_months'],
      dtype='object')
339
140


In [44]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0, max_depth = 50, min_samples_split = 0.01)
target = taxonomic_assignments['diet_milk'].fillna('weaned')
cross_val_score(clf, taxonomic_assignments[col_names_species], target, cv=10)

array([0.74850299, 0.61676647, 0.73053892, 0.74850299, 0.73652695,
       0.73652695, 0.79041916, 0.76646707, 0.73652695, 0.69277108])

In [38]:
taxonomic_assignments[col_names_species].isnull().sum().sum()

0